In [ ]:
@app.post("/generate_response")
async def generate_response(request: EmailRequest):
    graph = create_finance_agent_graph()
    initial_state = {
        "query": request.prompt,
        "step": "",
        "final_response": ""
        # Add other fields as necessary
    }

    async def stream_updates():
        current_state = initial_state
        async for chunk in graph.astream(current_state, stream_mode="updates"):
            for node, node_state in chunk.items():
                if isinstance(node_state, dict):
                    if node_state.get("step") != current_state.get("step"):
                        yield f"data: {json.dumps({'type': 'step', 'content': node_state.get('step')})}\n\n"
                    
                    if node_state.get("final_response") and not current_state.get("final_response"):
                        yield f"data: {json.dumps({'type': 'final', 'content': node_state.get('final_response')})}\n\n"
                        yield "data: [DONE]\n\n"
                        return
                    
                    current_state.update(node_state)
                else:
                    print(f"Unexpected state type for node {node}: {type(node_state)}")

    return StreamingResponse(stream_updates(), media_type="text/event-stream")

In [ ]:
@app.post("/generate_response")
async def generate_response(request: EmailRequest):
    graph = create_finance_agent_graph()
    initial_state = {
        "query": request.prompt,
        "step": "",
        "final_response": ""
        # Add other fields as necessary
    }

    async def stream_updates():
        current_state = initial_state
        async for chunk in graph.astream(current_state, stream_mode="updates"):
            for node, node_state in chunk.items():
                if isinstance(node_state, dict):
                    if node_state.get("step") != current_state.get("step"):
                        yield f"data: {json.dumps({'type': 'step', 'content': node_state.get('step')})}\n\n"
                    
                    if node_state.get("final_response") and not current_state.get("final_response"):
                        yield f"data: {json.dumps({'type': 'final', 'content': node_state.get('final_response')})}\n\n"
                        yield "data: [DONE]\n\n"
                        return
                    
                    current_state.update(node_state)
                else:
                    print(f"Unexpected state type for node {node}: {type(node_state)}")

    return StreamingResponse(stream_updates(), media_type="text/event-stream")

In [ ]:
  const handleAction = async (userInput) => {
    setIsProcessing(true);
    setError(null);
    setStatusMessage(null);
    setProcessingSteps([]);
    
    // Immediately add the user's message to the chat
    setChatMessages(prevMessages => [...prevMessages, { role: "user", content: userInput }]);
    
    try {
      const action = userConfig.buttons[0];
      console.log(`Handling action: ${action.label}, User input: ${userInput}`);
      const content = await getEmailContent();
      const payload = {
        userId: userConfig.userId,
        emailContent: content,
        prompt: userInput,
      };

      console.log(`Sending request to: ${action.apiEndpoint}`);
      const response = await fetch(action.apiEndpoint, {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify(payload),
      });

      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      
      const reader = response.body.getReader();
      const decoder = new TextDecoder();

      while (true) {
        const { value, done } = await reader.read();
        if (done) break;
        const chunk = decoder.decode(value);
        const lines = chunk.split('\n');
        
        for (const line of lines) {
          if (line.startsWith('data: ')) {
            const data = JSON.parse(line.slice(5));
            if (data === '[DONE]') {
              setIsProcessing(false);
              break;
            }
            handleUpdate(data);
          }
        }
      }
    } catch (e) {
      console.error(`Error in handleAction: ${e.message}`);
      setError(`Failed to process your request. Please try again.`);
      setIsProcessing(false);
    }
  };

  const handleUpdate = (data) => {
    if (data.type === 'step') {
      setProcessingSteps(prevSteps => [...prevSteps, data.content]);
    } else if (data.type === 'final') {
      setChatMessages(prevMessages => [
        ...prevMessages,
        { role: "assistant", content: data.content }
      ]);
      setProcessingSteps([]); // Clear steps after final response
      setIsProcessing(false);
    }
  };

  const handleChatSubmit = (e) => {
    e.preventDefault();
    if (chatInput.trim()) {
      handleAction(chatInput);
      setChatInput("");
    }
  };

  if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <div
              key={index}
              className={`${styles.chatMessage} ${
                message.role === "user" ? styles.userMessage : styles.assistantMessage
              }`}
            >
              {message.role === "user" ? (
                message.content
              ) : (
                <div dangerouslySetInnerHTML={{ __html: message.content }} />
              )}
            </div>
          ))}
        </div>
        {isProcessing && (
          <div className={styles.processingSteps}>
            {processingSteps.map((step, index) => (
              <div key={index} className={styles.processingStep}>
                {step}
              </div>
            ))}
          </div>
        )}
        <form onSubmit={handleChatSubmit} className={styles.chatInput}>
          <input
            type="text"
            value={chatInput}
            onChange={(e) => setChatInput(e.target.value)}
            placeholder="Type your message..."
            className={styles.chatInputField}
          />
          <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
            Send
          </Button>
        </form>
      </div>
    );
  }

  return (
    <div className={styles.container}>
      <div className={styles.actionBar} onClick={() => setIsChatActive(true)}>
        <div className={styles.actionContent}>
          <div style={{ display: "flex", alignItems: "center" }}>
            <div className={styles.iconWrapper}>
              <FluentIcons.MailTemplate24Regular />
            </div>
            <Text className={styles.actionTitle}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.actionDescription}>{userConfig.buttons[0].description}</Text>
        </div>
      </div>
    </div>
  );
};

export default EmailGenerator;